        # '''                           CS 637
                                      Homework - 2
                                   KUSHAL SHANKAR RAJ
                         Deep Learning with Pytorch on CIFAR10 Dataset




Design and Implementation:

In this assignment we'll be using PyTorch to construct a convolutional neural network. 

Our focus is to train the CNN on the CIFAR-10 data set to be able to classify images from the CIFAR-10 testing set into the ten categories present in the data set.

The CIFAR-10 data set is composed of 60,000 32x32 colour images, 6,000 images per class, so 10 categories in total. The training set is made up of 50,000 images, while the remaining 10,000 make up the testing set.

The categories are: airplane, automobile, bird, cat, deer, dog, frog, horse, ship and truck.

First step: Importing required libraries and extracting dataset

Next: Defining a Convolutional Neural network model.

The network has the following layout,

Input > Conv (ReLU) > MaxPool > Conv (ReLU) > MaxPool > FC (ReLU) > FC (ReLU) > FC (SoftMax) > 10 outputs

where:
Conv is a convolutional layer, ReLU is the activation function, MaxPool is a pooling layer, FC is a fully connected layer and SoftMax is the activation function of the output layer.

Next: Relu activation is used after convolution to scale features to greater than or equal to zero.

Second layer: Batch Normalization and Pooling. Role in reducing dimensions of the image, removing the
              irrelevant features and preserving the relevant.

Here i'm using a two layered convolutional neural network(2-D conv). Input is flattened before passing it to
the fully connected layer.

Next: Fully connected layer. Finally learning the necessary features of an image for image classification.

Finally: The loss is calculated and gradients are calculated along backward propagation.
Weights are updated accordingly for every iteration.

We are using Relu as activation function. 
Softmax at the output layer for classification and cross entropy as loss function.

The program is generalized in a way that the number of parameters of the neural network 
can be customised according the user’s flexibility. 

Various parameters are:

	Learning_rate and epochs: The learning rate and the number of iterations for training of the
                               test data can be passed as an argument.

	Batch_size: Required batch size to train the neural network to learn the parameters, in order to classify
                the images into ten classes.
                
Plot of Loss and Accuracies ar given below.'''

# Importing req. Libraries

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import math

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import _pickle as pickle
import numpy as np
import os
import sklearn.preprocessing as norm
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

# Data_Loader

In [2]:
def get_CIFAR10_data(cifar10_dir, num_training=49000, num_validation=1000, num_test=1000):
    '''
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the neural net classifier.
    '''
    # Load the raw CIFAR-10 data
    X_train, y_train, X_test, y_test = load(cifar10_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    X_train = X_train.astype(np.float64)
    X_val = X_val.astype(np.float64)
    X_test = X_test.astype(np.float64)

    # Transpose so that channels come first
    X_train = X_train.transpose(0, 3, 1, 2)
    X_val = X_val.transpose(0, 3, 1, 2)
    X_test = X_test.transpose(0, 3, 1, 2)
    mean_image = np.mean(X_train, axis=0)
    std = np.std(X_train)

    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    X_train /= std
    X_val /= std
    X_test /= std

    return {
        'X_train': X_train, 'y_train': y_train,
        'X_val': X_val, 'y_val': y_val,
        'X_test': X_test, 'y_test': y_test,
        'mean': mean_image, 'std': std
    }


def load_CIFAR_batch(filename):
    ''' load single batch of cifar '''
    with open(filename, 'rb') as f:
        datadict = pickle.load(f, encoding ='bytes')
        X = datadict[b'data']
        Y = datadict[b'labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0, 2, 3, 1)
        Y = np.array(Y)
        return X, Y


def load(ROOT):
    ''' load all of cifar '''
    xs = []
    ys = []
    for b in range(1, 6):
        f = os.path.join(ROOT, 'data_batch_%d' % (b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(ROOT, 'test_batch'))
    return Xtr, Ytr, Xte, Yte

In [3]:
Dict = {}
Dict = get_CIFAR10_data('cifar-10-batches-py/')

val_data = Dict['X_val']
val_labels = Dict['y_val']

test_data = Dict['X_test']
test_labels = Dict['y_test']

training_data = Dict['X_train']
training_labels = Dict['y_train']

mean = Dict['mean']
stand = Dict['std']

train_x  = torch.from_numpy(training_data).float()
train_y = training_labels.astype(int);
train_y = torch.from_numpy(train_y).float()

val_d  = torch.from_numpy(val_data).float()
val_l = val_labels.astype(int);
val_l = torch.from_numpy(val_l).float()

test_x  = torch.from_numpy(test_data).float()
test_y = test_labels.astype(int);
test_y = torch.from_numpy(test_y).float()

# CNN Model Training-Validation-Testing

In [4]:

#CNN model definition...
class Conv_Net(nn.Module):
    def __init__(self):
        super(Conv_Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5)
        self.nor1 = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=16, kernel_size=5)
        self.nor2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(in_features=(16 * 5 * 5), out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=80)
        self.fc3 = nn.Linear(in_features=80, out_features=10)

        #Forward propag..  
    def forward(self, x):
        x = self.pool(F.relu(self.nor1(self.conv1(x))))
        x = self.pool(F.relu(self.nor2(self.conv2(x))))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def training_loss(self,traincal):
        plt.plot(traincal)
        plt.xlabel("No: of Iterations")
        plt.ylabel("Cost")
        plt.title('----Training Loss----')
        plt.show()
        
    def train_acc(self,accurate):
        plt.plot(accurate)
        plt.xlabel("No: of Iterations")
        plt.ylabel("Accuracy")
        plt.title('----Training Accuracy----')
        plt.show()
            
    def validation_loss(self,validcal):
        plt.plot(validcal)
        plt.xlabel("No: of Iterations")
        plt.ylabel("Cost")
        plt.title('----Validation Loss----')
        plt.show()
   
    # MODEL TRAINING AND VALIDATION
    
    def Model_Training(self,train_x,train_y,val_d,val_l,test_x,test_y,Learning_rate,batch_size,epochs):
        Learning_rate = Learning_rate
        batch_size = batch_size
        epochs = epochs
        net = Conv_Net()
        
        #Training the network. 
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=Learning_rate, momentum=0.9)
        print('\n\n--------------Reports - Plot & Evaluation------------------\n')
        iter = int(len(train_x)/batch_size) 

        loss_traincal = []
        loss_validcal = []
        train_accuracy = []

        net = net.float()
        training_loss = 0.0
        validation_loss = 0.0

        for epoch in range(epochs):

            k = 0
            training_loss = 0.0
            validation_loss = 0.0
            correct_count = 0
            total_count = 0

            for i in range(0, iter):

                train_data = train_x[k: k+batch_size]
                train_label = train_y[k: k+batch_size]

                #print(i," : ",train_data.shape)
                inputs = train_data
                labels = train_label.long()

                optimizer.zero_grad()

                outputs = net(inputs)
                
                _, predicted = torch.max(outputs.data, 1)
                total_count += labels.size(0)
                correct_count += (predicted == labels).sum().item()
                
                loss = criterion(outputs, labels)

                training_loss += loss

                loss.backward()

                optimizer.step()

                k += batch_size

                if(i==(iter-1)):
                    p = 0
                    batch = 8
                    iterater = int(len(val_d)/batch) 

                    #print('Validation epoch: ',epoch+1)
                    for w in range(0, iterater):

                        validation_d = val_d[p: p+batch]
                        validation_l = val_l[p: p+batch]

                        #print(i," : ",train_data.shape)
                        input_s = validation_d
                        label = validation_l

                        optimizer.zero_grad()

                        output = net(input_s)

                        loss = criterion(output, label.long())

                        validation_loss += loss

                        loss.backward()

                        optimizer.step()

                        p += batch

                    loss_validcal.append(validation_loss)
                    #print('End of validation')

            loss_traincal.append(training_loss)
             
            accu = (100 * correct_count)/total_count
            train_accuracy.append(accu)
            v = math.ceil((accu*100)/100)
            print('Training Accuracy of the network @ Epoch ',epoch+1,' := ',v,'%')
            print('----------------------------------------------')
        
        self.training_loss(loss_traincal)
        self.train_acc(train_accuracy)
        self.validation_loss(loss_validcal)
        
        batchss = 16
        iteration = int(len(test_x)/batchss) 
        correct = 0
        total = 0
        with torch.no_grad():
            t = 0
            for j in range(0, iteration):
                test = test_x[t: t+batchss]
                test_lab1 = test_y[t: t+batchss]

                test_lab = test_lab1.long()

                out = net(test)
                _, predicted = torch.max(out.data, 1)
                total += test_lab.size(0)
                correct += (predicted == test_lab).sum().item()
                t += batchss
        
        print('\n----------------------------------------------\n')
        print('Accuracy of the network on the 1000 test images: %d %%' % (100 * correct / total))
        print('\n----------------------------------------------\n')
        
        bat_size = 16
        it = int(len(test_x)/bat_size) 
        classes = ('plane', 'car', 'bird', 'cat',
                   'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
        class_correct = list(0. for i in range(10))
        class_total = list(0. for i in range(10))
        with torch.no_grad():
            q = 0
            for j in range(0, it):
                test = test_x[q: q+bat_size]
                test_lab = test_y[q: q+bat_size]

                test_lab = test_lab.long()

                out_put = net(test)
                _, predicted = torch.max(out_put, 1)
                c = (predicted == test_lab).squeeze()
                for i in range(4):
                    label = test_lab[i]
                    class_correct[label] += c[i].item()
                    class_total[label] += 1
                q += bat_size

        print('Accuracies for Given Classes........\n')
        for i in range(10):
            print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))

# Model Training and Reports

In [ ]:
net = Conv_Net()

print("\n\n------Cifar-10 Data Classification Model-------\n")
print("Enter Learning Rate... (Recommmended 0.001)   : ") 
Learning_rate = float(input()) 
print("Enter Batch Size....    (Recommmended 32)     : ") 
Batch_Size = int(input()) 
print("Enter No:of Epochs....  (Recommmended 40)     : ") 
Iterations_epoch = int(input())

net.Model_Training(train_x,train_y,val_d,val_l,test_x,test_y,Learning_rate,Batch_Size,Iterations_epoch)



------Cifar-10 Data Classification Model-------

Enter Learning Rate... (Recommmended 0.001)   : 
